In [0]:
import datetime

In [0]:
current_date_str = datetime.datetime.now().strftime("%d%m%Y")
print(current_date_str)

In [0]:
jdbc_hostname = "j2dtechnologies.database.windows.net"
jdbc_port = 1433
jdbc_database = "j2ddatabase"
base_path =  "/mnt/my_adls_container/samples/sql_tables"

In [0]:
# check the scret scopes

dbutils.secrets.listScopes()

In [0]:
try:  
    user_name = dbutils.secrets.get(scope="sc-jd2tech-05", key="azure-sql-user-name")
    password = dbutils.secrets.get(scope="sc-jd2tech-05", key="azure-sql-password")
except Exception as e:
    raise e

In [0]:
# build jdbc url
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};database={jdbc_database}"

connection_properties = {
    "user": user_name,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
query = "(select * from INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE') as temp"

tables_info_df = (
    spark.read.jdbc(
        url=jdbc_url,
        table=query,
        properties=connection_properties
    )
)
display(tables_info_df)

In [0]:
def sql_to_adls(tables_info):
    for row in tables_info.collect():
        schema = row["TABLE_SCHEMA"]
        table_name = row["TABLE_NAME"]
        full_table_name = f"{schema}.{table_name}"

        print(f"Processing table: {full_table_name}")

        try:
            df = (
                spark.read.jdbc(
                    url=jdbc_url,
                    table=f"{schema}.{table_name}",
                    properties=connection_properties
                )
            )

            # write into adls
            (
                df.write.format("parquet")
                .mode("overwrite")
                .save(f"{base_path}/{((full_table_name.replace('.', '_')).lower())}_{current_date_str}")
            )
        
        except Exception as e:
            raise e

In [0]:
# function call
sql_to_adls(tables_info_df)

In [0]:
dbutils.fs.mounts()